## SSC Case Study: Initial data staging 

In [2]:
import pandas as pd
import numpy as np
import time

In [6]:
incidents = pd.read_csv("ssc2021_case_study_cer.csv")
incidents_extend = pd.read_csv("2021-03-31-comprehensive-incident-data.csv")
dates = pd.read_csv("Dates_2008_2020.csv")

In [4]:
incidents.head()

,Event Number,Occurred/Discovered Date and Time,Product Category,Product Type,Volume Released,Latitude,Longitude
0,INC2008-001,2008-01-02 11:30,Liquid,Crude Oil - Sweet,8.0,49.73135,-101.235570
1,INC2008-004,2008-01-23 6:05,Liquid,Crude Oil - Sweet,100.0,49.73135,-101.235570
2,INC2008-008,2008-01-26 10:45,Gas,Natural Gas - Sweet,NaN,58.01200,-122.694333
3,INC2008-009,2008-01-27 12:01,Miscellaneous,Potassium Hydroxide (caustic solution),NaN,56.14377,-120.516050
4,INC2008-128,2008-02-04 12:00,Gas,Natural Gas - Sweet,NaN,49.76492,-84.918240


In [5]:
incidents_extend.head()

,Incident Number,Incident Types,Reported Date,Nearest Populated Centre,Province,Company,Status,Latitude,Approximate Volume Released (m3),Substance,...,Repair type,Repair date,Equipment or component has never been inspected,Most recent inspection date for the failed equipment or component,Type of most recent inspection,Most recent inspection part of the routine inspection program,No maintenance done on this equipment or component,Date of the most recent maintenance work for the failed equipment or component,Most recent maintenance Type,Most recent maintenance work part of the routine maintenance program
0,INC2008-001,Release of Substance,1/2/2008,Cromer,Manitoba,Enbridge Pipelines Inc.,Closed,49.731350,8,Crude Oil - Sweet,...,Type B Sleeve (Pressure Containing),1/11/2008,No,NaN,NaN,No,No,1/11/2008,Repair,No
1,INC2008-004,Release of Substance,1/23/2008,Cromer,Manitoba,Enbridge Pipelines Inc.,Closed,49.731350,100,Crude Oil - Sweet,...,Replacement,1/24/2008,No,NaN,NaN,No,No,NaN,Replacement,No
2,INC2008-008,Release of Substance,1/29/2008,Fort Nelson,British Columbia,"Westcoast Energy Inc., carrying on business as...",Closed,58.012000,Not Provided,Natural Gas - Sweet,...,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No
3,INC2008-009,"Release of Substance, Adverse Environmental Ef...",1/29/2008,Taylor,British Columbia,"Westcoast Energy Inc., carrying on business as...",Closed,56.143770,Not Provided,Potassium Hydroxide (caustic solution),...,NaN,NaN,No,NaN,NaN,No,No,NaN,NaN,No
4,INC2008-016,Release of Substance,2/23/2008,Beiseker,Alberta,Foothills Pipe Lines Ltd.,Closed,51.385029,Not Provided,Natural Gas - Sweet,...,Rebuilt,3/25/2008,No,NaN,NaN,No,No,3/25/2008,Repair,No


In [7]:
dates.head()

,Date_Key,Date_Full_Format,Day,Month,Year,Day_of_Week_Name,Day_of_Week_Num,Week_in_Year,Weekend
0,1.0,2008-01-01,1.0,1.0,2008.0,Tuesday,3.0,1.0,0.0
1,2.0,2008-01-02,2.0,1.0,2008.0,Wednesday,4.0,1.0,0.0
2,3.0,2008-01-03,3.0,1.0,2008.0,Thursday,5.0,1.0,0.0
3,4.0,2008-01-04,4.0,1.0,2008.0,Friday,6.0,1.0,0.0
4,5.0,2008-01-05,5.0,1.0,2008.0,Saturday,7.0,1.0,1.0
